# Linear Programming with Python

## Overview

Linear programming (LP) is the workhorse of power system optimization. From economic dispatch to transmission planning, many critical decisions can be formulated as linear programs and solved efficiently even at massive scale. This lesson provides hands-on experience implementing and solving LP problems using Python.

We'll explore multiple Python libraries for optimization, understand their strengths and trade-offs, and develop skills for debugging optimization models. By the end, you'll be able to confidently translate power system problems into code and extract meaningful insights from the solutions.

## Learning Objectives

By completing this lesson, you will be able to:

1. Implement linear programs using multiple Python libraries
2. Choose appropriate tools for different problem types and scales
3. Debug common issues: infeasibility, unboundedness, numerical problems
4. Extract and interpret both primal and dual solutions
5. Build reusable optimization models for power system applications
6. Understand solver behavior and performance considerations

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import linprog
import warnings
warnings.filterwarnings('ignore')

# We'll install PuLP if not available
try:
    import pulp
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pulp"])
    import pulp

print("Libraries loaded successfully")
print(f"PuLP version: {pulp.__version__}")
print(f"SciPy version: {np.__version__}")

## 1. Linear Programming Review

### Standard Form

Linear programs have the general form:

**Minimize:** $c^T x$

**Subject to:**
- $A_{eq} x = b_{eq}$ (equality constraints)
- $A_{ub} x \leq b_{ub}$ (inequality constraints)  
- $l \leq x \leq u$ (variable bounds)

Where:
- $x$ is the vector of decision variables
- $c$ is the cost coefficient vector
- $A$ and $b$ define the constraints

### Power System Context

Many power system problems are naturally linear:
- **Economic dispatch** with linear costs
- **DC optimal power flow** (linearized power flow)
- **Transmission planning** with fixed costs per line
- **Market clearing** with supply and demand bids

Let's start with our familiar economic dispatch problem.

In [ ]:
# Economic Dispatch Example
# 3 generators must meet 400 MW demand

# Generator data
gen_data = pd.DataFrame({
    'Generator': ['G1', 'G2', 'G3'],
    'Pmin (MW)': [50, 40, 60],
    'Pmax (MW)': [200, 150, 180],
    'Cost ($/MWh)': [25, 30, 28]
})

demand = 400  # MW

print("Economic Dispatch Problem:")
print(gen_data)
print(f"\nSystem demand: {demand} MW")
print("\nMathematical formulation:")
print("Minimize: 25*P1 + 30*P2 + 28*P3")
print("Subject to:")
print("  P1 + P2 + P3 = 400")
print("  50 ≤ P1 ≤ 200")
print("  40 ≤ P2 ≤ 150")
print("  60 ≤ P3 ≤ 180")

## 2. Introduction to Python LP Solvers

Python offers several libraries for solving linear programs. Each has strengths and use cases:

### SciPy's linprog
- **Pros**: Built into SciPy, no extra installation, good for learning
- **Cons**: Limited features, requires matrix form, basic solver
- **Use for**: Simple problems, teaching, when dependencies are limited

### PuLP
- **Pros**: Pythonic syntax, multiple solver backends, widely used
- **Cons**: Slower model building for huge problems
- **Use for**: Most power system optimization problems

### Pyomo
- **Pros**: Very powerful, supports nonlinear, great for research
- **Cons**: Steeper learning curve, more complex syntax
- **Use for**: Complex models, multi-stage problems, when you need advanced features

### CVXPY
- **Pros**: Elegant mathematical syntax, automatic problem classification
- **Cons**: Focused on convex optimization, may be overkill for simple LPs
- **Use for**: When transitioning between linear and nonlinear convex problems

Let's solve our economic dispatch problem using SciPy first.

In [ ]:
# Solving with SciPy's linprog
# Convert to standard form: minimize c^T x subject to A_eq x = b_eq, bounds

# Objective coefficients
c = np.array([25, 30, 28])  # Cost coefficients

# Equality constraint: P1 + P2 + P3 = 400
A_eq = np.array([[1, 1, 1]])
b_eq = np.array([400])

# Bounds for each generator
bounds = [(50, 200),   # G1
          (40, 150),   # G2  
          (60, 180)]   # G3

# Solve
result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# Display results
print("SciPy Solution:")
print(f"Status: {result.message}")
print(f"Success: {result.success}")
print(f"\nOptimal generation:")
for i, gen in enumerate(gen_data['Generator']):
    print(f"  {gen}: {result.x[i]:.1f} MW")
print(f"\nTotal cost: ${result.fun:.2f}/hour")

# Verify constraints
total_gen = sum(result.x)
print(f"\nVerification:")
print(f"Total generation: {total_gen:.1f} MW (should be {demand})")
print(f"All generators within limits: {all(bounds[i][0] <= result.x[i] <= bounds[i][1] for i in range(3))}")

### Understanding the Solution

Notice that the cheapest generator (G1 at $25/MWh) is at its maximum output (200 MW). This is the expected economic behavior - use cheaper resources first. The remaining demand is split between G2 and G3 based on their costs.

## 3. Building LP Models with PuLP

PuLP provides a more intuitive interface for building optimization models. Instead of matrices, you work with named variables and constraints, making the code more readable and maintainable.

In [ ]:
# Economic dispatch with PuLP
# Create the problem
prob = pulp.LpProblem("Economic_Dispatch", pulp.LpMinimize)

# Decision variables
# Create generator variables with bounds
P1 = pulp.LpVariable("P1", lowBound=50, upBound=200)
P2 = pulp.LpVariable("P2", lowBound=40, upBound=150)
P3 = pulp.LpVariable("P3", lowBound=60, upBound=180)

# Objective function
prob += 25*P1 + 30*P2 + 28*P3, "Total_Cost"

# Constraints
prob += P1 + P2 + P3 == 400, "Power_Balance"

# Solve the problem
prob.solve()

# Display results
print("PuLP Solution:")
print(f"Status: {pulp.LpStatus[prob.status]}")
print(f"\nOptimal generation:")
print(f"  G1: {P1.varValue:.1f} MW")
print(f"  G2: {P2.varValue:.1f} MW")
print(f"  G3: {P3.varValue:.1f} MW")
print(f"\nTotal cost: ${pulp.value(prob.objective):.2f}/hour")

# Show the constraint dual value (shadow price)
print(f"\nShadow price of power balance: ${prob.constraints['Power_Balance'].pi:.2f}/MW")
print("(This is the marginal cost of the system)")

### Building More Complex Models

Let's solve a multi-period economic dispatch with ramping constraints. This demonstrates PuLP's strength in handling indexed variables and multiple constraints.

In [ ]:
# Multi-period economic dispatch with ramping constraints
# 4 time periods, 3 generators

# Problem data
periods = range(4)
generators = range(3)

# Extended generator data
costs = [25, 30, 28]        # $/MWh
p_min = [50, 40, 60]        # MW
p_max = [200, 150, 180]     # MW
ramp_limit = [50, 40, 60]   # MW/period

# Time-varying demand
demand = [250, 300, 280, 260]  # MW

# Create problem
prob = pulp.LpProblem("Multi_Period_Dispatch", pulp.LpMinimize)

# Decision variables - use dictionaries for indexed variables
p = {}
for g in generators:
    for t in periods:
        p[g,t] = pulp.LpVariable(f"P_g{g}_t{t}", p_min[g], p_max[g])

# Objective function - total cost over all periods
prob += pulp.lpSum(costs[g] * p[g,t] for g in generators for t in periods)

# Constraints
# Power balance for each period
for t in periods:
    prob += pulp.lpSum(p[g,t] for g in generators) == demand[t], f"Balance_t{t}"

# Ramping constraints
for g in generators:
    for t in periods[1:]:  # Starting from period 1
        # Ramp up limit
        prob += p[g,t] - p[g,t-1] <= ramp_limit[g], f"RampUp_g{g}_t{t}"
        # Ramp down limit  
        prob += p[g,t-1] - p[g,t] <= ramp_limit[g], f"RampDown_g{g}_t{t}"

# Solve
prob.solve()

# Display results in a nice table
print(f"Multi-Period Dispatch Solution")
print(f"Status: {pulp.LpStatus[prob.status]}")
print(f"Total cost: ${pulp.value(prob.objective):.2f}")
print("\nDispatch Schedule:")
print("Period\tDemand\tG1\tG2\tG3\tTotal")
print("-" * 50)

for t in periods:
    gen_outputs = [p[g,t].varValue for g in generators]
    total = sum(gen_outputs)
    print(f"{t}\t{demand[t]}\t{gen_outputs[0]:.0f}\t{gen_outputs[1]:.0f}\t{gen_outputs[2]:.0f}\t{total:.0f}")

# Check ramping
print("\nRamp rates (MW/period):")
print("Period\tG1\tG2\tG3")
print("-" * 30)
for t in periods[1:]:
    ramps = [p[g,t].varValue - p[g,t-1].varValue for g in generators]
    print(f"{t-1}→{t}\t{ramps[0]:+.0f}\t{ramps[1]:+.0f}\t{ramps[2]:+.0f}")

```{admonition} Exercise 1: Add Reserve Requirements
:class: tip

Modify the multi-period dispatch to include spinning reserve requirements:
- Each period needs 10% of demand as spinning reserve
- Spinning reserve = available headroom on running generators
- Add the constraints and resolve

How does this change the dispatch and total cost?
```

In [ ]:
# Exercise 1: Your solution here
# Add reserve requirements to the multi-period dispatch

# Your solution:

In [ ]:
# Solution
# Multi-period dispatch with reserves
prob_reserve = pulp.LpProblem("Multi_Period_Dispatch_with_Reserve", pulp.LpMinimize)

# Decision variables
p_r = {}
for g in generators:
    for t in periods:
        p_r[g,t] = pulp.LpVariable(f"P_g{g}_t{t}", p_min[g], p_max[g])

# Objective function
prob_reserve += pulp.lpSum(costs[g] * p_r[g,t] for g in generators for t in periods)

# Power balance constraints
for t in periods:
    prob_reserve += pulp.lpSum(p_r[g,t] for g in generators) == demand[t], f"Balance_t{t}"

# Reserve constraints - 10% of demand
for t in periods:
    reserve_req = 0.1 * demand[t]
    # Available headroom = sum of (Pmax - P) for all generators
    prob_reserve += pulp.lpSum(p_max[g] - p_r[g,t] for g in generators) >= reserve_req, f"Reserve_t{t}"

# Ramping constraints
for g in generators:
    for t in periods[1:]:
        prob_reserve += p_r[g,t] - p_r[g,t-1] <= ramp_limit[g], f"RampUp_g{g}_t{t}"
        prob_reserve += p_r[g,t-1] - p_r[g,t] <= ramp_limit[g], f"RampDown_g{g}_t{t}"

# Solve
prob_reserve.solve()

# Compare results
print("Impact of Reserve Requirements:")
print("=" * 50)
print(f"Without reserves - Total cost: ${pulp.value(prob.objective):.2f}")
print(f"With reserves - Total cost: ${pulp.value(prob_reserve.objective):.2f}")
print(f"Cost increase: ${pulp.value(prob_reserve.objective) - pulp.value(prob.objective):.2f}")

print("\nDispatch Comparison:")
print("Period\tDemand\tReserve\t| No Reserve (MW)\t| With Reserve (MW)")
print("\t\tReq\t| G1   G2   G3\t\t| G1   G2   G3")
print("-" * 70)

for t in periods:
    # Without reserve
    gen_no_res = [p[g,t].varValue for g in generators]
    # With reserve
    gen_with_res = [p_r[g,t].varValue for g in generators]
    reserve_req = 0.1 * demand[t]
    
    print(f"{t}\t{demand[t]}\t{reserve_req:.0f}\t| ", end="")
    print(f"{gen_no_res[0]:.0f}  {gen_no_res[1]:.0f}  {gen_no_res[2]:.0f}\t| ", end="")
    print(f"{gen_with_res[0]:.0f}  {gen_with_res[1]:.0f}  {gen_with_res[2]:.0f}")

# Check reserve provision
print("\nReserve Provision:")
print("Period\tRequired\tAvailable\tMargin")
for t in periods:
    required = 0.1 * demand[t]
    available = sum(p_max[g] - p_r[g,t].varValue for g in generators)
    print(f"{t}\t{required:.1f}\t\t{available:.1f}\t\t{available-required:.1f}")

print("\nKey Insight: Reserve requirements prevent generators from running at maximum,")
print("increasing costs as more expensive units must run to maintain headroom.")

## 4. Debugging Optimization Models

Real-world optimization models often have issues that need debugging. Common problems include:

1. **Infeasibility**: No solution satisfies all constraints
2. **Unboundedness**: Objective can be improved infinitely
3. **Numerical issues**: Poor scaling or near-singular matrices
4. **Unexpected solutions**: Model doesn't capture intended behavior

Let's explore debugging techniques with intentionally problematic models.

In [ ]:
# Example: Debugging an infeasible model
# Demand is too high for available generation

prob_infeas = pulp.LpProblem("Infeasible_Dispatch", pulp.LpMinimize)

# Same generators as before
P1_inf = pulp.LpVariable("P1", lowBound=50, upBound=200)
P2_inf = pulp.LpVariable("P2", lowBound=40, upBound=150)
P3_inf = pulp.LpVariable("P3", lowBound=60, upBound=180)

# Objective
prob_infeas += 25*P1_inf + 30*P2_inf + 28*P3_inf

# Infeasible constraint - demand exceeds maximum generation
max_gen = 200 + 150 + 180  # 530 MW
infeasible_demand = 550  # MW

prob_infeas += P1_inf + P2_inf + P3_inf == infeasible_demand, "Power_Balance"

# Try to solve
prob_infeas.solve()

print(f"Problem status: {pulp.LpStatus[prob_infeas.status]}")
print(f"\nDemand: {infeasible_demand} MW")
print(f"Maximum possible generation: {max_gen} MW")
print(f"Shortage: {infeasible_demand - max_gen} MW")

# Debugging approach 1: Check constraint feasibility
print("\nDebugging Approach 1: Check individual constraints")
print(f"Minimum generation: {50 + 40 + 60} MW")
print(f"Maximum generation: {200 + 150 + 180} MW")
print(f"Can we meet demand? {50 + 40 + 60 <= infeasible_demand <= 200 + 150 + 180}")

In [ ]:
# Debugging approach 2: Relax constraints to find conflicts
# Add slack variables to make problem feasible

prob_debug = pulp.LpProblem("Debug_Dispatch", pulp.LpMinimize)

# Original variables
P1_d = pulp.LpVariable("P1", lowBound=50, upBound=200)
P2_d = pulp.LpVariable("P2", lowBound=40, upBound=150)
P3_d = pulp.LpVariable("P3", lowBound=60, upBound=180)

# Slack variables for constraint violation
slack_pos = pulp.LpVariable("Slack_positive", lowBound=0)  # Over-generation
slack_neg = pulp.LpVariable("Slack_negative", lowBound=0)  # Under-generation

# Objective: minimize cost + heavy penalty for slack
penalty = 10000  # High penalty for constraint violation
prob_debug += (25*P1_d + 30*P2_d + 28*P3_d + 
               penalty*slack_pos + penalty*slack_neg)

# Modified power balance with slack
prob_debug += P1_d + P2_d + P3_d + slack_neg - slack_pos == infeasible_demand

# Solve
prob_debug.solve()

print("Debugging with slack variables:")
print(f"Status: {pulp.LpStatus[prob_debug.status]}")
print(f"\nGeneration:")
print(f"  P1: {P1_d.varValue:.1f} MW")
print(f"  P2: {P2_d.varValue:.1f} MW") 
print(f"  P3: {P3_d.varValue:.1f} MW")
print(f"  Total: {P1_d.varValue + P2_d.varValue + P3_d.varValue:.1f} MW")
print(f"\nSlack variables:")
print(f"  Under-generation (load shed): {slack_neg.varValue:.1f} MW")
print(f"  Over-generation (curtailment): {slack_pos.varValue:.1f} MW")

if slack_neg.varValue > 0.1:
    print(f"\nConclusion: System is short {slack_neg.varValue:.1f} MW")
    print("Solutions: 1) Reduce demand, 2) Add generation, 3) Implement load shedding")

```{admonition} Exercise 2: Debug a Transmission-Constrained Problem
:class: tip

The following dispatch problem includes transmission constraints but returns an unexpected solution. Debug the model to find and fix the issue.

System:
- 3 generators at different buses
- 3 loads at the same buses
- Transmission limits between buses
- The model seems to curtail cheap generation - why?
```

In [ ]:
# Exercise 2: Debug this transmission-constrained dispatch
# Something is wrong - the solution doesn't make economic sense

prob_trans = pulp.LpProblem("Transmission_Dispatch", pulp.LpMinimize)

# Three buses with generation and load
buses = range(3)
gen_cost = [20, 30, 25]  # $/MWh
gen_max = [150, 100, 120]  # MW
load = [80, 90, 100]  # MW at each bus

# Transmission limits (from bus i to bus j)
trans_limit = {
    (0,1): 50,
    (1,0): 50,
    (1,2): 40,
    (2,1): 40,
    (0,2): 30,
    (2,0): 30
}

# Variables
gen = {}
flow = {}

for b in buses:
    gen[b] = pulp.LpVariable(f"Gen_{b}", lowBound=0, upBound=gen_max[b])

for i in buses:
    for j in buses:
        if i != j:
            flow[i,j] = pulp.LpVariable(f"Flow_{i}_{j}", 
                                       lowBound=-trans_limit.get((i,j), 0),
                                       upBound=trans_limit.get((i,j), 0))

# Objective
prob_trans += pulp.lpSum(gen_cost[b] * gen[b] for b in buses)

# Power balance at each bus
for b in buses:
    # Generation - Load + Net Imports = 0
    # BUG: There's an error in this constraint formulation
    prob_trans += (gen[b] - load[b] + 
                  pulp.lpSum(flow[i,b] for i in buses if i != b) -
                  pulp.lpSum(flow[b,j] for j in buses if j != b) == 0, 
                  f"Balance_bus_{b}")

# Solve and display
prob_trans.solve()

print("Transmission-Constrained Dispatch:")
print(f"Status: {pulp.LpStatus[prob_trans.status]}")
print(f"\nGeneration (Cost):")
total_gen = 0
for b in buses:
    total_gen += gen[b].varValue
    print(f"  Bus {b}: {gen[b].varValue:.1f} MW (${gen_cost[b]}/MWh)")
print(f"\nTotal generation: {total_gen:.1f} MW")
print(f"Total load: {sum(load)} MW")
print(f"\nSomething seems wrong - cheapest generator (Bus 0) is not at maximum!")

# Your debugging code here:

In [ ]:
# Solution: Debug the transmission-constrained problem

# First, let's check the flows to understand what's happening
print("\nDEBUGGING ANALYSIS:")
print("=" * 50)

print("\n1. Check power flows:")
for i in buses:
    for j in buses:
        if i != j and (i,j) in flow:
            if abs(flow[i,j].varValue) > 0.1:
                print(f"  Flow {i}→{j}: {flow[i,j].varValue:.1f} MW (limit: {trans_limit.get((i,j), 0)})") 

print("\n2. Check power balance at each bus:")
for b in buses:
    net_export = sum(flow[b,j].varValue for j in buses if j != b and (b,j) in flow)
    net_import = sum(flow[i,b].varValue for i in buses if i != b and (i,b) in flow)
    balance = gen[b].varValue - load[b] - net_export + net_import
    print(f"  Bus {b}: Gen={gen[b].varValue:.1f}, Load={load[b]}, Export={net_export:.1f}, Import={net_import:.1f}, Balance={balance:.1f}")

print("\n3. THE BUG: The flow variables are defined with wrong bounds!")
print("   Each flow[i,j] can be negative (reverse flow) but we want:")
print("   - flow[0,1] represents flow from bus 0 to bus 1")
print("   - This should be bounded by [0, trans_limit]")
print("   - We need separate variables for each direction")

# FIXED VERSION
print("\n" + "=" * 50)
print("CORRECTED MODEL:")
print("=" * 50)

prob_fixed = pulp.LpProblem("Fixed_Transmission_Dispatch", pulp.LpMinimize)

# Variables
gen_f = {}
flow_f = {}

for b in buses:
    gen_f[b] = pulp.LpVariable(f"Gen_{b}", lowBound=0, upBound=gen_max[b])

# Create flow variables only for allowed directions with correct bounds
for (i,j), limit in trans_limit.items():
    flow_f[i,j] = pulp.LpVariable(f"Flow_{i}_{j}", lowBound=0, upBound=limit)

# Objective
prob_fixed += pulp.lpSum(gen_cost[b] * gen_f[b] for b in buses)

# Power balance at each bus (corrected)
for b in buses:
    # Generation - Load - Exports + Imports = 0
    exports = pulp.lpSum(flow_f[b,j] for j in buses if (b,j) in flow_f)
    imports = pulp.lpSum(flow_f[i,b] for i in buses if (i,b) in flow_f)
    
    prob_fixed += gen_f[b] - load[b] - exports + imports == 0, f"Balance_bus_{b}"

# Solve
prob_fixed.solve()

print("\nCorrected Solution:")
print(f"Status: {pulp.LpStatus[prob_fixed.status]}")

print(f"\nGeneration:")
for b in buses:
    print(f"  Bus {b}: {gen_f[b].varValue:.1f} MW (${gen_cost[b]}/MWh)")

print(f"\nFlows:")
for (i,j) in flow_f:
    if flow_f[i,j].varValue > 0.1:
        print(f"  {i}→{j}: {flow_f[i,j].varValue:.1f} MW")

print(f"\nTotal cost: ${pulp.value(prob_fixed.objective):.2f}")

print("\nNow the cheap generator at Bus 0 is used more!")
print("The transmission limits still prevent full utilization,")
print("but the dispatch is now economically sensible.")

# Visualize the solution
import matplotlib.patches as patches

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Original (buggy) solution
ax = ax1
ax.set_title("Original (Buggy) Solution")
bus_pos = [(1, 2), (0, 0), (2, 0)]

for b, (x, y) in enumerate(bus_pos):
    # Draw bus
    circle = patches.Circle((x, y), 0.3, facecolor='lightblue', edgecolor='black', linewidth=2)
    ax.add_patch(circle)
    ax.text(x, y, f"Bus {b}", ha='center', va='center', fontweight='bold')
    
    # Show generation and load
    ax.text(x, y+0.5, f"G: {gen[b].varValue:.0f} MW\n${gen_cost[b]}/MWh", 
            ha='center', va='bottom', bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen"))
    ax.text(x, y-0.5, f"L: {load[b]} MW", ha='center', va='top',
            bbox=dict(boxstyle="round,pad=0.3", facecolor="lightcoral"))

ax.set_xlim(-0.5, 2.5)
ax.set_ylim(-1, 3)
ax.set_aspect('equal')
ax.axis('off')

# Corrected solution
ax = ax2
ax.set_title("Corrected Solution")

for b, (x, y) in enumerate(bus_pos):
    # Draw bus
    circle = patches.Circle((x, y), 0.3, facecolor='lightblue', edgecolor='black', linewidth=2)
    ax.add_patch(circle)
    ax.text(x, y, f"Bus {b}", ha='center', va='center', fontweight='bold')
    
    # Show generation and load
    ax.text(x, y+0.5, f"G: {gen_f[b].varValue:.0f} MW\n${gen_cost[b]}/MWh", 
            ha='center', va='bottom', bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen"))
    ax.text(x, y-0.5, f"L: {load[b]} MW", ha='center', va='top',
            bbox=dict(boxstyle="round,pad=0.3", facecolor="lightcoral"))

# Draw flows for corrected solution
for (i,j) in flow_f:
    if flow_f[i,j].varValue > 0.1:
        x1, y1 = bus_pos[i]
        x2, y2 = bus_pos[j]
        ax.annotate('', xy=(x2, y2), xytext=(x1, y1),
                   arrowprops=dict(arrowstyle='->', lw=2, color='red'))
        # Label the flow
        mx, my = (x1+x2)/2, (y1+y2)/2
        ax.text(mx, my, f"{flow_f[i,j].varValue:.0f} MW", 
               bbox=dict(boxstyle="round,pad=0.2", facecolor="yellow"))

ax.set_xlim(-0.5, 2.5) 
ax.set_ylim(-1, 3)
ax.set_aspect('equal')
ax.axis('off')

plt.tight_layout()
plt.show()

## 5. Analyzing Results

Optimization solvers provide more than just the optimal values of decision variables. Understanding all available information helps in:
- Validating solutions
- Performing sensitivity analysis  
- Identifying bottlenecks
- Making better decisions

### Primal and Dual Solutions

- **Primal solution**: Values of decision variables (e.g., generator outputs)
- **Dual solution**: Shadow prices of constraints (marginal values)
- **Reduced costs**: How much objective would change if we forced a variable to change
- **Slack values**: How much "room" remains in inequality constraints

In [ ]:
# Comprehensive result analysis
# Let's create a more complex problem to analyze

# System with transmission constraints and losses
prob_analysis = pulp.LpProblem("Comprehensive_Analysis", pulp.LpMinimize)

# 4 generators with different characteristics
gen_data = pd.DataFrame({
    'Gen': ['Nuclear', 'Coal', 'Gas', 'Wind'],
    'Bus': [0, 1, 2, 3],
    'Min_MW': [100, 50, 0, 0],
    'Max_MW': [100, 200, 150, 80],
    'Cost_$/MWh': [10, 25, 40, 0],
    'Must_Run': [True, False, False, False]
})

# Load at different buses
load_data = pd.DataFrame({
    'Bus': [0, 1, 2, 3],
    'Load_MW': [50, 120, 80, 30]
})

# Simplified transmission (adjacent buses connected)
lines = [(0,1), (1,2), (2,3)]
line_limit = 100  # MW on each line

# Variables
gen_vars = {}
for idx, g in gen_data.iterrows():
    gen_vars[g['Gen']] = pulp.LpVariable(g['Gen'], 
                                         lowBound=g['Min_MW'], 
                                         upBound=g['Max_MW'])

flow_vars = {}
for line in lines:
    # Bidirectional flows
    flow_vars[line] = pulp.LpVariable(f"Flow_{line[0]}_{line[1]}", 
                                     lowBound=-line_limit, 
                                     upBound=line_limit)

# Objective: minimize generation cost
prob_analysis += pulp.lpSum(gen_data.loc[gen_data['Gen']==g, 'Cost_$/MWh'].values[0] * gen_vars[g] 
                           for g in gen_vars)

# Constraints
# Power balance at each bus
for bus in range(4):
    # Generation at this bus
    gen_at_bus = pulp.lpSum(gen_vars[g] for g in gen_data[gen_data['Bus']==bus]['Gen'])
    
    # Load at this bus
    load_at_bus = load_data[load_data['Bus']==bus]['Load_MW'].values[0]
    
    # Net flow (positive = export)
    flow_out = pulp.lpSum(flow_vars[line] for line in lines if line[0]==bus)
    flow_in = pulp.lpSum(-flow_vars[line] for line in lines if line[1]==bus)
    
    prob_analysis += gen_at_bus - load_at_bus - flow_out - flow_in == 0, f"Balance_Bus_{bus}"

# Must-run constraint for nuclear
prob_analysis += gen_vars['Nuclear'] == 100, "Nuclear_Must_Run"

# Wind availability constraint (forecast)
wind_available = 60
prob_analysis += gen_vars['Wind'] <= wind_available, "Wind_Availability"

# Solve
prob_analysis.solve()

print("SOLUTION ANALYSIS")
print("=" * 60)
print(f"Status: {pulp.LpStatus[prob_analysis.status]}")
print(f"Total Cost: ${pulp.value(prob_analysis.objective):.2f}")

# Primal solution
print("\n1. PRIMAL SOLUTION (Generation):")
print("Generator\tOutput (MW)\tCost ($/MWh)\tTotal Cost ($)")
print("-" * 60)
total_gen = 0
for g in gen_vars:
    output = gen_vars[g].varValue
    cost_rate = gen_data.loc[gen_data['Gen']==g, 'Cost_$/MWh'].values[0]
    total_gen += output
    print(f"{g}\t\t{output:.1f}\t\t{cost_rate}\t\t{output * cost_rate:.2f}")
print(f"\nTotal Generation: {total_gen:.1f} MW")
print(f"Total Load: {load_data['Load_MW'].sum()} MW")

# Flow solution
print("\n2. TRANSMISSION FLOWS:")
for line in lines:
    flow = flow_vars[line].varValue
    utilization = abs(flow) / line_limit * 100
    print(f"Line {line[0]}→{line[1]}: {flow:+.1f} MW ({utilization:.1f}% utilized)")

# Dual solution (shadow prices)
print("\n3. DUAL SOLUTION (Shadow Prices):")
print("Constraint\t\t\tShadow Price\tInterpretation")
print("-" * 70)

for bus in range(4):
    constraint_name = f"Balance_Bus_{bus}"
    shadow_price = prob_analysis.constraints[constraint_name].pi
    print(f"{constraint_name}\t\t{-shadow_price:.2f} $/MW\tLMP at Bus {bus}")

# Special constraints
print(f"\nWind_Availability\t\t{-prob_analysis.constraints['Wind_Availability'].pi:.2f} $/MW", end="")
print("\tValue of additional wind")

# Reduced costs
print("\n4. REDUCED COSTS:")
print("(How much the objective would improve if we forced the variable to change)")
for g in gen_vars:
    rc = gen_vars[g].dj
    if abs(rc) > 0.01:
        print(f"{g}: {rc:.2f} $/MW")

# Sensitivity analysis
print("\n5. SENSITIVITY INSIGHTS:")
lmps = [-prob_analysis.constraints[f"Balance_Bus_{bus}"].pi for bus in range(4)]
max_lmp = max(lmps)
min_lmp = min(lmps)
print(f"LMP range: ${min_lmp:.2f} - ${max_lmp:.2f}/MWh")
print(f"Congestion rent: ${(max_lmp - min_lmp) * min(100, total_gen/4):.2f}/hour")

if prob_analysis.constraints['Wind_Availability'].pi < -0.01:
    print(f"\nWind is valuable! Each additional MW saves ${-prob_analysis.constraints['Wind_Availability'].pi:.2f}/hour")

```{admonition} Exercise 3: Sensitivity Analysis
:class: tip

Using the solution above:
1. What happens to total cost if we increase transmission capacity by 10 MW?
2. Which generator would be most valuable to expand by 50 MW?
3. At what gas price would gas generation go to zero?

Hint: Use shadow prices and re-solve with parameter changes.
```

In [ ]:
# Exercise 3: Your solution here
# Perform sensitivity analysis

# Your solution:

In [ ]:
# Solution: Sensitivity Analysis

print("SENSITIVITY ANALYSIS")
print("=" * 60)

# Save original solution for comparison
original_cost = pulp.value(prob_analysis.objective)
original_gen = {g: gen_vars[g].varValue for g in gen_vars}

# 1. Impact of transmission expansion
print("\n1. TRANSMISSION EXPANSION ANALYSIS:")
print("Increase all line limits by 10 MW")

# Create new problem with increased limits
prob_trans_exp = pulp.LpProblem("Transmission_Expansion", pulp.LpMinimize)

# Same variables but with higher line limits
gen_vars_exp = {}
for idx, g in gen_data.iterrows():
    gen_vars_exp[g['Gen']] = pulp.LpVariable(g['Gen'], 
                                             lowBound=g['Min_MW'], 
                                             upBound=g['Max_MW'])

flow_vars_exp = {}
new_limit = line_limit + 10  # Increased by 10 MW
for line in lines:
    flow_vars_exp[line] = pulp.LpVariable(f"Flow_{line[0]}_{line[1]}", 
                                          lowBound=-new_limit, 
                                          upBound=new_limit)

# Same objective
prob_trans_exp += pulp.lpSum(gen_data.loc[gen_data['Gen']==g, 'Cost_$/MWh'].values[0] * gen_vars_exp[g] 
                             for g in gen_vars_exp)

# Same constraints with new variables
for bus in range(4):
    gen_at_bus = pulp.lpSum(gen_vars_exp[g] for g in gen_data[gen_data['Bus']==bus]['Gen'])
    load_at_bus = load_data[load_data['Bus']==bus]['Load_MW'].values[0]
    flow_out = pulp.lpSum(flow_vars_exp[line] for line in lines if line[0]==bus)
    flow_in = pulp.lpSum(-flow_vars_exp[line] for line in lines if line[1]==bus)
    prob_trans_exp += gen_at_bus - load_at_bus - flow_out - flow_in == 0, f"Balance_Bus_{bus}"

prob_trans_exp += gen_vars_exp['Nuclear'] == 100, "Nuclear_Must_Run"
prob_trans_exp += gen_vars_exp['Wind'] <= wind_available, "Wind_Availability"

prob_trans_exp.solve()

new_cost = pulp.value(prob_trans_exp.objective)
savings = original_cost - new_cost

print(f"Original cost: ${original_cost:.2f}")
print(f"New cost: ${new_cost:.2f}")
print(f"Savings: ${savings:.2f}/hour")
print(f"Annual savings (8760 hours): ${savings * 8760:,.0f}")

# Show generation changes
print("\nGeneration changes:")
for g in gen_vars:
    old = original_gen[g]
    new = gen_vars_exp[g].varValue
    if abs(new - old) > 0.1:
        print(f"{g}: {old:.1f} → {new:.1f} MW (change: {new-old:+.1f} MW)")

# 2. Generator expansion analysis
print("\n2. GENERATOR EXPANSION ANALYSIS:")
print("Expand each generator by 50 MW and check cost reduction")

expansion_value = {}
for gen_to_expand in ['Coal', 'Gas']:  # Nuclear is must-run, Wind is free
    # Create problem with expanded generator
    prob_gen_exp = pulp.LpProblem(f"Expand_{gen_to_expand}", pulp.LpMinimize)
    
    gen_vars_gen_exp = {}
    for idx, g in gen_data.iterrows():
        if g['Gen'] == gen_to_expand:
            # Increase max by 50 MW
            gen_vars_gen_exp[g['Gen']] = pulp.LpVariable(g['Gen'], 
                                                         lowBound=g['Min_MW'], 
                                                         upBound=g['Max_MW'] + 50)
        else:
            gen_vars_gen_exp[g['Gen']] = pulp.LpVariable(g['Gen'], 
                                                         lowBound=g['Min_MW'], 
                                                         upBound=g['Max_MW'])
    
    # Rest of the problem setup (abbreviated)
    flow_vars_gen_exp = {line: pulp.LpVariable(f"Flow_{line[0]}_{line[1]}", 
                                               lowBound=-line_limit, upBound=line_limit) 
                         for line in lines}
    
    prob_gen_exp += pulp.lpSum(gen_data.loc[gen_data['Gen']==g, 'Cost_$/MWh'].values[0] * gen_vars_gen_exp[g] 
                               for g in gen_vars_gen_exp)
    
    for bus in range(4):
        gen_at_bus = pulp.lpSum(gen_vars_gen_exp[g] for g in gen_data[gen_data['Bus']==bus]['Gen'])
        load_at_bus = load_data[load_data['Bus']==bus]['Load_MW'].values[0]
        flow_out = pulp.lpSum(flow_vars_gen_exp[line] for line in lines if line[0]==bus)
        flow_in = pulp.lpSum(-flow_vars_gen_exp[line] for line in lines if line[1]==bus)
        prob_gen_exp += gen_at_bus - load_at_bus - flow_out - flow_in == 0, f"Balance_Bus_{bus}"
    
    prob_gen_exp += gen_vars_gen_exp['Nuclear'] == 100, "Nuclear_Must_Run"
    prob_gen_exp += gen_vars_gen_exp['Wind'] <= wind_available, "Wind_Availability"
    
    prob_gen_exp.solve()
    
    exp_cost = pulp.value(prob_gen_exp.objective)
    expansion_value[gen_to_expand] = original_cost - exp_cost
    
    print(f"\nExpanding {gen_to_expand} by 50 MW:")
    print(f"  Cost savings: ${expansion_value[gen_to_expand]:.2f}/hour")
    print(f"  New {gen_to_expand} output: {gen_vars_gen_exp[gen_to_expand].varValue:.1f} MW")

best_expansion = max(expansion_value, key=expansion_value.get)
print(f"\nBest generator to expand: {best_expansion}")
print(f"Value: ${expansion_value[best_expansion]:.2f}/hour")

# 3. Gas price sensitivity
print("\n3. GAS PRICE SENSITIVITY:")
print("Find gas price where gas generation goes to zero")

gas_prices = range(25, 60, 5)  # Test different gas prices
gas_generation = []

for gas_price in gas_prices:
    # Create problem with new gas price
    prob_gas = pulp.LpProblem(f"Gas_Price_{gas_price}", pulp.LpMinimize)
    
    gen_vars_gas = {}
    for idx, g in gen_data.iterrows():
        gen_vars_gas[g['Gen']] = pulp.LpVariable(g['Gen'], 
                                                 lowBound=g['Min_MW'], 
                                                 upBound=g['Max_MW'])
    
    flow_vars_gas = {line: pulp.LpVariable(f"Flow_{line[0]}_{line[1]}", 
                                           lowBound=-line_limit, upBound=line_limit) 
                     for line in lines}
    
    # Modified objective with new gas price
    prob_gas += pulp.lpSum(
        (gas_price if g == 'Gas' else gen_data.loc[gen_data['Gen']==g, 'Cost_$/MWh'].values[0]) * gen_vars_gas[g]
        for g in gen_vars_gas
    )
    
    # Constraints (same as before)
    for bus in range(4):
        gen_at_bus = pulp.lpSum(gen_vars_gas[g] for g in gen_data[gen_data['Bus']==bus]['Gen'])
        load_at_bus = load_data[load_data['Bus']==bus]['Load_MW'].values[0]
        flow_out = pulp.lpSum(flow_vars_gas[line] for line in lines if line[0]==bus)
        flow_in = pulp.lpSum(-flow_vars_gas[line] for line in lines if line[1]==bus)
        prob_gas += gen_at_bus - load_at_bus - flow_out - flow_in == 0, f"Balance_Bus_{bus}"
    
    prob_gas += gen_vars_gas['Nuclear'] == 100, "Nuclear_Must_Run"
    prob_gas += gen_vars_gas['Wind'] <= wind_available, "Wind_Availability"
    
    prob_gas.solve()
    
    gas_gen = gen_vars_gas['Gas'].varValue
    gas_generation.append(gas_gen)
    
    print(f"Gas price ${gas_price}/MWh → Gas generation: {gas_gen:.1f} MW")
    
    if gas_gen < 0.1:
        print(f"\nGas generation goes to zero at ${gas_price}/MWh")
        break

# Plot gas generation vs price
plt.figure(figsize=(8, 5))
plt.plot(gas_prices[:len(gas_generation)], gas_generation, 'b-o', linewidth=2, markersize=8)
plt.xlabel('Gas Price ($/MWh)')
plt.ylabel('Gas Generation (MW)')
plt.title('Gas Generation vs Gas Price')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='red', linestyle='--', alpha=0.5)

# Add original point
plt.plot(40, original_gen['Gas'], 'r*', markersize=15, label='Original')
plt.legend()

plt.tight_layout()
plt.show()

print("\nKEY INSIGHTS:")
print(f"1. Transmission expansion saves ${savings:.2f}/hour by enabling cheaper generation")
print(f"2. {best_expansion} expansion is most valuable due to its cost and location")
print(f"3. Gas becomes uneconomic when its price exceeds the next available resource")

## 6. Performance Considerations

As problems grow in size, performance becomes crucial. Real power system models can have millions of variables and constraints.

### Model Building Performance

- Use vectorized operations when possible
- Pre-calculate parameters outside the model
- Avoid nested loops in constraint generation
- Consider sparse representations

### Solver Performance

- **Open-source solvers**: CBC (good), GLPK (basic), HiGHS (excellent)
- **Commercial solvers**: Gurobi, CPLEX (much faster for large problems)
- **Warm starting**: Use previous solutions as starting points
- **Presolve**: Let solver simplify before solving
- **Parallel solving**: Modern solvers use multiple cores

In [ ]:
# Performance comparison example
import time

# Create a larger problem - DC OPF for N-bus system
def create_dc_opf(n_buses, n_lines):
    """Create a DC OPF problem of given size"""
    prob = pulp.LpProblem(f"DC_OPF_{n_buses}_bus", pulp.LpMinimize)
    
    # Random system data
    np.random.seed(42)
    gen_buses = np.random.choice(n_buses, size=n_buses//3, replace=False)
    gen_cost = np.random.uniform(20, 50, size=len(gen_buses))
    gen_max = np.random.uniform(50, 200, size=len(gen_buses))
    
    loads = np.random.uniform(20, 100, size=n_buses)
    loads[gen_buses] *= 0.5  # Reduce load at generator buses
    
    # Variables
    gen = {}
    for i, bus in enumerate(gen_buses):
        gen[bus] = pulp.LpVariable(f"Gen_{bus}", 0, gen_max[i])
    
    # Objective
    prob += pulp.lpSum(gen_cost[i] * gen[bus] for i, bus in enumerate(gen_buses))
    
    # Simple power balance (ignoring transmission for this example)
    prob += pulp.lpSum(gen[bus] for bus in gen_buses) == sum(loads), "Power_Balance"
    
    return prob

# Test different problem sizes
sizes = [10, 50, 100, 200]
results = []

print("Performance Testing:")
print("Buses\tBuild Time\tSolve Time\tTotal Time")
print("-" * 50)

for n in sizes:
    # Model building time
    start = time.time()
    prob = create_dc_opf(n, n*2)
    build_time = time.time() - start
    
    # Solving time
    start = time.time()
    prob.solve(pulp.PULP_CBC_CMD(msg=0))  # Suppress output
    solve_time = time.time() - start
    
    total_time = build_time + solve_time
    results.append((n, build_time, solve_time, total_time))
    
    print(f"{n}\t{build_time:.3f}s\t\t{solve_time:.3f}s\t\t{total_time:.3f}s")

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Build vs solve time
buses = [r[0] for r in results]
build_times = [r[1] for r in results]
solve_times = [r[2] for r in results]

ax1.plot(buses, build_times, 'b-o', label='Model Building', linewidth=2)
ax1.plot(buses, solve_times, 'r-s', label='Solving', linewidth=2)
ax1.set_xlabel('Number of Buses')
ax1.set_ylabel('Time (seconds)')
ax1.set_title('Model Building vs Solving Time')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')

# Solver comparison (simulated)
solvers = ['CBC', 'GLPK', 'HiGHS', 'Gurobi*']
relative_times = [1.0, 2.5, 0.8, 0.1]  # Relative to CBC
colors = ['blue', 'orange', 'green', 'red']

x = np.arange(len(solvers))
ax2.bar(x, relative_times, color=colors)
ax2.set_xlabel('Solver')
ax2.set_ylabel('Relative Solution Time')
ax2.set_title('Solver Performance Comparison (1000-bus system)')
ax2.set_xticks(x)
ax2.set_xticklabels(solvers)
ax2.axhline(y=1, color='black', linestyle='--', alpha=0.5)
ax2.text(3, 0.15, '*Commercial', ha='center', style='italic')

plt.tight_layout()
plt.show()

print("\nPerformance Tips:")
print("1. Model building time grows with problem complexity")
print("2. Solver time depends on problem structure, not just size")
print("3. Commercial solvers can be 10-100x faster for large problems")
print("4. Consider decomposition for very large systems")

```{admonition} Exercise 4: Build a Reusable ED Function
:class: tip

Create a reusable function for economic dispatch that:
1. Takes generator data and demand as inputs
2. Handles any number of generators
3. Returns dispatch, total cost, and LMPs
4. Includes optional transmission constraints

Test it with different system configurations.
```

In [ ]:
# Exercise 4: Your solution here
# Create a reusable economic dispatch function

def economic_dispatch(gen_data, demand, transmission=None):
    """
    Solve economic dispatch problem
    
    Parameters:
    gen_data: DataFrame with columns ['name', 'bus', 'min_mw', 'max_mw', 'cost']
    demand: Total system demand (MW) or dict of bus demands
    transmission: Optional dict of line limits
    
    Returns:
    dict with 'dispatch', 'cost', 'lmps', 'status'
    """
    # Your implementation here
    pass

# Test your function

In [ ]:
# Solution: Reusable Economic Dispatch Function

def economic_dispatch(gen_data, demand, transmission=None):
    """
    Solve economic dispatch problem
    
    Parameters:
    gen_data: DataFrame with columns ['name', 'bus', 'min_mw', 'max_mw', 'cost']
    demand: Total system demand (MW) or dict of bus demands
    transmission: Optional DataFrame with columns ['from', 'to', 'limit']
    
    Returns:
    dict with 'dispatch', 'cost', 'lmps', 'flows', 'status'
    """
    
    # Create problem
    prob = pulp.LpProblem("Economic_Dispatch", pulp.LpMinimize)
    
    # Determine if we have bus-specific demands
    if isinstance(demand, dict):
        buses = list(set(gen_data['bus'].unique()) | set(demand.keys()))
        bus_demand = demand
    else:
        buses = gen_data['bus'].unique()
        # Distribute demand proportionally or put all at bus 0
        bus_demand = {buses[0]: demand}
    
    # Decision variables
    gen_vars = {}
    for idx, gen in gen_data.iterrows():
        gen_vars[gen['name']] = pulp.LpVariable(
            gen['name'], 
            lowBound=gen['min_mw'], 
            upBound=gen['max_mw']
        )
    
    # Transmission variables if needed
    flow_vars = {}
    if transmission is not None:
        for idx, line in transmission.iterrows():
            flow_vars[(line['from'], line['to'])] = pulp.LpVariable(
                f"Flow_{line['from']}_{line['to']}",
                lowBound=-line['limit'],
                upBound=line['limit']
            )
    
    # Objective: minimize total cost
    prob += pulp.lpSum(gen['cost'] * gen_vars[gen['name']] 
                      for idx, gen in gen_data.iterrows())
    
    # Constraints
    if transmission is None:
        # Simple system-wide balance
        prob += (pulp.lpSum(gen_vars[gen['name']] for idx, gen in gen_data.iterrows()) == 
                sum(bus_demand.values())), "System_Balance"
    else:
        # Bus-wise balance with transmission
        for bus in buses:
            # Generation at this bus
            gen_at_bus = pulp.lpSum(gen_vars[gen['name']] 
                                   for idx, gen in gen_data.iterrows() 
                                   if gen['bus'] == bus)
            
            # Demand at this bus
            demand_at_bus = bus_demand.get(bus, 0)
            
            # Net exports
            exports = pulp.lpSum(flow_vars.get((bus, other_bus), 0) 
                               for other_bus in buses if bus != other_bus)
            
            # Net imports  
            imports = pulp.lpSum(flow_vars.get((other_bus, bus), 0)
                               for other_bus in buses if bus != other_bus)
            
            prob += (gen_at_bus - demand_at_bus - exports + imports == 0, 
                    f"Balance_Bus_{bus}")
    
    # Solve
    prob.solve()
    
    # Extract results
    results = {
        'status': pulp.LpStatus[prob.status],
        'dispatch': {},
        'cost': pulp.value(prob.objective) if prob.status == 1 else None,
        'lmps': {},
        'flows': {}
    }
    
    if prob.status == 1:  # Optimal
        # Generation dispatch
        for gen_name, var in gen_vars.items():
            results['dispatch'][gen_name] = var.varValue
        
        # LMPs (shadow prices)
        if transmission is None:
            # Single system price
            results['lmps']['system'] = -prob.constraints['System_Balance'].pi
        else:
            # Bus-specific LMPs
            for bus in buses:
                constraint_name = f"Balance_Bus_{bus}"
                if constraint_name in prob.constraints:
                    results['lmps'][bus] = -prob.constraints[constraint_name].pi
        
        # Flows
        for (f, t), var in flow_vars.items():
            if abs(var.varValue) > 0.1:
                results['flows'][(f, t)] = var.varValue
    
    return results

# Test the function
print("Testing Reusable Economic Dispatch Function")
print("=" * 50)

# Test 1: Simple system
print("\nTest 1: Simple 3-generator system")
simple_gens = pd.DataFrame({
    'name': ['G1', 'G2', 'G3'],
    'bus': [0, 0, 0],
    'min_mw': [50, 40, 60],
    'max_mw': [200, 150, 180],
    'cost': [25, 30, 28]
})

result1 = economic_dispatch(simple_gens, 400)
print(f"Status: {result1['status']}")
print(f"Total cost: ${result1['cost']:.2f}")
print(f"System LMP: ${result1['lmps']['system']:.2f}/MWh")
print("Dispatch:", {k: f"{v:.1f} MW" for k, v in result1['dispatch'].items()})

# Test 2: Multi-bus with transmission
print("\nTest 2: 3-bus system with transmission constraints")
bus_gens = pd.DataFrame({
    'name': ['G1', 'G2', 'G3'],
    'bus': [0, 1, 2],
    'min_mw': [0, 0, 0],
    'max_mw': [150, 100, 120],
    'cost': [20, 35, 25]
})

trans_data = pd.DataFrame({
    'from': [0, 1, 0],
    'to': [1, 2, 2],
    'limit': [80, 60, 100]
})

bus_loads = {0: 50, 1: 100, 2: 80}

result2 = economic_dispatch(bus_gens, bus_loads, trans_data)
print(f"Status: {result2['status']}")
print(f"Total cost: ${result2['cost']:.2f}")
print("\nLMPs by bus:")
for bus, lmp in result2['lmps'].items():
    print(f"  Bus {bus}: ${lmp:.2f}/MWh")
print("\nDispatch:")
for gen, mw in result2['dispatch'].items():
    bus = bus_gens[bus_gens['name']==gen]['bus'].values[0]
    print(f"  {gen} (Bus {bus}): {mw:.1f} MW")
print("\nFlows:")
for (f, t), flow in result2['flows'].items():
    print(f"  {f} → {t}: {flow:.1f} MW")

# Test 3: Infeasible case
print("\nTest 3: Infeasible demand")
result3 = economic_dispatch(simple_gens, 600)  # Too high
print(f"Status: {result3['status']}")
print(f"Cost: {result3['cost']}")

print("\nFunction successfully handles various system configurations!")

## Summary

This lesson provided hands-on experience with linear programming in Python, focusing on power system applications. Key takeaways:

### Tools and Libraries
- **SciPy** provides basic LP solving but requires matrix formulation
- **PuLP** offers intuitive model building with named variables and constraints
- Choose tools based on problem complexity and team familiarity
- Commercial solvers provide significant performance benefits for large problems

### Model Building Best Practices
- Start simple and incrementally add complexity
- Use meaningful variable and constraint names
- Validate data before building models
- Structure code for reusability

### Debugging Techniques
- Check feasibility by examining constraint conflicts
- Use slack variables to diagnose infeasibility
- Verify units and sign conventions
- Visualize solutions when possible

### Solution Analysis
- Primal values give optimal decisions
- Dual values (shadow prices) indicate constraint values
- Sensitivity analysis guides system improvements
- Always verify solutions make physical sense

### Performance Considerations
- Model building time can exceed solving time
- Problem structure matters more than size
- Consider decomposition for very large systems
- Warm starting helps for repeated solves

## Next Steps

With these LP skills established, you're ready to tackle more complex power system optimization problems. The next lesson explores how these techniques apply to electricity markets, where optimization determines prices and dispatch in competitive environments.

Remember: behind every market price and dispatch decision is an optimization problem being solved. Understanding these fundamentals helps you interpret market outcomes and design better systems.